In [1]:
from datasets import load_dataset
import openai
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

/opt/anaconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_excel('/Users/jd/trauma_llm/trauma_data/updated_embedding_responses.xlsx')

total_response = 3

res = dataset['Value 1']
for ii in range(total_response-1):
    res += dataset['Value '+str(ii+2)]
res /= total_response

dataset['score'] = res

In [3]:
dataset.head()


,Response 1,Response 2,Value 1,Value 2,Value 3,score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,7,2,3,2.333333
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,11,4,4,3.666667
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,11,5,4,3.666667
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...",8,2,4,2.666667
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...",12,5,4,4.000000


In [12]:
# Load dataset
#dataset = load_dataset("sentence-transformers/stsb", split="train")
'''dataset = load_dataset("tabilab/biosses", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")'''


'dataset = load_dataset("tabilab/biosses", split="train")\nprint(f"Loaded {len(dataset)} examples. Sample:")\nprint(dataset[0], "\n")'

In [4]:
# Set your OpenAI API key (best via environment variable)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Function to get embedding
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        model=model,
        input=[text]
    )
    # Extract the embedding vector
    return response.data[0].embedding

In [6]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
file_to_save = 'trauma/openai_text-embedding-3-large.pickle'
embeddings = pd.DataFrame()
sentence1 = []
sentence2 = []
embedding1 = []
embedding2 = []
cosine_similarity = []
human_score = []

print('Extracting embedding and measuring similarity ...')
time.sleep(1)
for idx in tqdm(range(len(dataset))):
    first_sentence = dataset["Response 1"][idx]
    emb1 = get_embedding(first_sentence)
    sentence1.append(
        first_sentence
    )
    embedding1.append(
        emb1
    )
    #############################
    second_sentence = dataset["Response 2"][idx]
    emb2 = get_embedding(second_sentence)
    sentence2.append(
        second_sentence
    )
    embedding2.append(
        emb2
    )
    
    ############################
    cosine_similarity.append(
        1 - cosine(emb1, emb2)
    )
    human_score.append(
        dataset["score"][idx]/5.0
    )
    
    time.sleep(1)  # respect rate limits!

embeddings['Sentence 1'] = sentence1
embeddings['Sentence 2'] = sentence2
embeddings['Embedding of Sentence 1'] = embedding1
embeddings['Embedding of Sentence 2'] = embedding2
embeddings['Cosine similarity'] = cosine_similarity
embeddings['Human score'] = human_score

with open(file_to_save, 'wb') as f:
    pickle.dump(embeddings, f)

Extracting embedding and measuring similarity ...


100%|██████████| 234/234 [07:37<00:00,  1.95s/it]


In [7]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,"[0.03468029946088791, 0.023846067488193512, -0...","[0.027717940509319305, 0.020076334476470947, -...",0.651570,0.466667
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,"[0.00306615955196321, -1.2684868124779314e-05,...","[0.021354706957936287, -0.009206251241266727, ...",0.599480,0.733333
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,"[0.006595144979655743, 0.012422056868672371, 0...","[-0.007823437452316284, -0.012772772461175919,...",0.415569,0.733333
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...","[0.024144506081938744, 0.011739034205675125, -...","[0.012490262277424335, 0.007518191821873188, -...",0.719269,0.533333
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...","[0.007118304260075092, -0.0008773026638664305,...","[-0.007297666277736425, -0.015604357235133648,...",0.567206,0.800000
